In [1]:
import networkx as nx
import obonet
import torch
import pandas as pd
import numpy as np

/tmp/ipykernel_3332239/1957606864.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
edges_df = pd.read_csv("GNN/go_protein_edges.csv")
features_df = pd.read_csv("GNN/go_protein_features.csv")
features_df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
features_df

,index,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature759,feature760,feature761,feature762,feature763,feature764,feature765,feature766,feature767,feature768
0,GO:0000001,-1.168093,-0.355214,0.265877,-0.710051,0.515028,-0.525165,-0.186588,-0.161192,0.186984,...,-1.350874,-0.991801,-0.648123,-0.361629,-0.914965,-0.506993,0.389760,0.207266,0.070705,0.938593
1,GO:0000002,-1.185879,-0.098765,0.388240,-0.295556,0.327296,-0.119842,0.399882,-0.035890,0.853417,...,-1.086927,-0.842870,-0.385764,0.175797,-1.223772,-0.999628,0.101473,-0.051212,0.048775,0.780470
2,GO:0000003,0.063323,-0.199995,0.151511,-0.942141,0.109313,0.015316,0.633298,0.507875,0.665548,...,0.174185,0.351648,0.138497,0.119273,-0.295167,-0.331179,0.102570,-0.524301,-0.139264,0.761573
3,GO:0000005,0.163135,0.301527,0.219680,0.094342,-0.129769,0.225696,0.357577,0.819992,0.852388,...,-0.084025,-0.291103,-0.003621,0.245929,-0.443244,0.229245,-0.685159,-0.725621,0.285964,0.313211
4,GO:0000006,-0.641113,-0.541363,0.413941,0.699345,0.461507,-0.497388,-0.044589,-0.655766,-0.596647,...,-0.561434,0.246475,-0.029871,-0.212828,-0.985273,0.677472,0.582681,0.299317,-0.131577,0.739702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69872,MIR29B2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
69873,PLXNA2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
69874,CELA2A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
69875,CELA2B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [3]:
labels_df = pd.read_csv('GNN/solubility.csv')
labels_df.rename(columns={'Gene name': 'protein'}, inplace=True)
labels_df.dropna()

,protein,Solubility,Label,Word_Count,Count_Category
0,ERAP2,Membrane,0,117,High Count
1,ADAMTSL5,Soluble,1,28,Low Count
2,TBC1D30,Membrane,0,55,High Count
3,KCNK18,Membrane,0,184,High Count
4,NDNF,Soluble,1,129,High Count
...,...,...,...,...,...
1374,TRABD2B,Membrane,0,96,High Count
1375,RPS9,Soluble,1,205,High Count
1376,SLC22A16,Membrane,0,93,High Count
1377,FBN3,Soluble,1,90,High Count


In [4]:
nodes_in_labels = set(labels_df['protein'])
nodes_in_features = set(features_df['index'])
common_nodes_labels = nodes_in_labels.intersection(nodes_in_features)
print(len(common_nodes_labels))
# 过滤
filtered_labels_df = labels_df[labels_df['protein'].isin(common_nodes_labels)].reset_index(drop=True)
print(len(filtered_labels_df))

1379
1379


In [5]:
labels_df.set_index('protein', inplace=True)
features_df.set_index('index', inplace=True)

In [6]:
node_labels = []

for node in features_df.index:
    if node in labels_df.index:
        node_labels.append(labels_df.loc[node, 'Label'])
    else:
        node_labels.append(-1)

labels_tensor = torch.tensor(node_labels, dtype=torch.long)
print(node_labels)

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,

In [7]:
node_labels = np.array(node_labels)
count_label_0 = np.count_nonzero(node_labels == 0)
count_label_1 = np.count_nonzero(node_labels == 1)

print("标签为 0 的节点数量：", count_label_0)
print("标签为 1 的节点数量：", count_label_1)

标签为 0 的节点数量： 670
标签为 1 的节点数量： 709


In [8]:
from torch_geometric.data import Data
node_index_map = {node_name: i for i, node_name in enumerate(features_df.index)}

node_features = torch.tensor(features_df.values, dtype=torch.float)
source_indices = [node_index_map[node] for node in edges_df['Source']]
target_indices = [node_index_map[node] for node in edges_df['Target']]
edge_index = torch.tensor([source_indices, target_indices], dtype=torch.long)

data = Data(x=node_features, edge_index=edge_index, y = labels_tensor)

In [9]:

print("x:", data.x.shape, data.x.dtype)
print("edge_index:", data.edge_index.shape, data.edge_index.dtype)
print("labels:", data.y.shape, data.y.dtype)

x: torch.Size([69877, 768]) torch.float32
edge_index: torch.Size([2, 477028]) torch.int64
labels: torch.Size([69877]) torch.int64


In [10]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.nn import Linear, ModuleList, Dropout

class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes, num_layers, activation, dropout):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.convs = ModuleList([GCNConv(hidden_dim, hidden_dim) for _ in range(num_layers - 2)])
        self.conv_last = GCNConv(hidden_dim, num_classes)
        self.activation = activation
        self.dropout = Dropout(dropout)
        self.num_layers = num_layers

    def forward(self, x, edge_index):
        # 输入层
        x = self.conv1(x, edge_index)
        x = self.activation(x)
        x = self.dropout(x)
        
        # 隐藏层
        for conv in self.convs:
            x = conv(x, edge_index)
            x = self.activation(x)
            x = self.dropout(x)

        # 输出层
        x = self.conv_last(x, edge_index)
        return x


In [11]:
from __future__ import division

import torch


def accuracy(pred, target):
    r"""Computes the accuracy of correct predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.

    :rtype: int
    """
    return (pred == target).sum().item() / target.numel()



def true_positive(pred, target, num_classes):
    r"""Computes the number of true positive predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred == i) & (target == i)).sum())

    return torch.tensor(out)



def true_negative(pred, target, num_classes):
    r"""Computes the number of true negative predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred != i) & (target != i)).sum())

    return torch.tensor(out)



def false_positive(pred, target, num_classes):
    r"""Computes the number of false positive predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred == i) & (target != i)).sum())

    return torch.tensor(out)



def false_negative(pred, target, num_classes):
    r"""Computes the number of false negative predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred != i) & (target == i)).sum())

    return torch.tensor(out)



def precision(pred, target, num_classes):
    r"""Computes the precision:
    :math:`\frac{\mathrm{TP}}{\mathrm{TP}+\mathrm{FP}}`.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`Tensor`
    """
    tp = true_positive(pred, target, num_classes).to(torch.float)
    fp = false_positive(pred, target, num_classes).to(torch.float)

    out = tp / (tp + fp)
    out[torch.isnan(out)] = 0

    return out



def recall(pred, target, num_classes):
    r"""Computes the recall:
    :math:`\frac{\mathrm{TP}}{\mathrm{TP}+\mathrm{FN}}`.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`Tensor`
    """
    tp = true_positive(pred, target, num_classes).to(torch.float)
    fn = false_negative(pred, target, num_classes).to(torch.float)

    out = tp / (tp + fn)
    out[torch.isnan(out)] = 0

    return out



def f1_score(pred, target, num_classes):
    r"""Computes the :math:`F_1` score:
    :math:`2 \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}
    {\mathrm{precision}+\mathrm{recall}}`.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`Tensor`
    """
    prec = precision(pred, target, num_classes)
    rec = recall(pred, target, num_classes)

    score = 2 * (prec * rec) / (prec + rec)
    score[torch.isnan(score)] = 0

    return score

In [12]:
from sklearn.preprocessing import label_binarize
def train_model_scheduler(model, masked_features, labels, edge_index, optimizer, criterion, scheduler, train_mask):
    model.train()  # 设置模型为训练模
    optimizer.zero_grad()  # 清空梯度
    out = model(masked_features, edge_index)  # 获取模型输出
    loss = criterion(out[train_mask], data.y[train_mask])  # 计算损失值，只针对训练集的节点
    loss.backward()  # 反向传播
    optimizer.step()  # 更新模型参数
    scheduler.step(loss)
    return loss.item()

def train_model(model, masked_features, labels, edge_index, optimizer, criterion, train_mask):
    model.train()  # 设置模型为训练模式
    optimizer.zero_grad()  # 清空梯度
    out = model(masked_features, edge_index) # 获取模型输出
    loss = criterion(out[train_mask], data.y[train_mask])  # 计算损失值，只针对训练集的节点
    loss.backward()  # 反向传播
    optimizer.step()  # 更新模型参数
    return loss.item()

def evaluate_model(model, features, labels, edge_index, mask):
    model.eval()  # 设置模型为评估模式
    with torch.no_grad():  # 关闭梯度计算
        # 获取模型输出，这里假设输出已经是经过sigmoid的概率
        probabilities = model(features, edge_index)
        if probabilities.shape[1] == 2:  # 假设有两个输出（每个类一个概率）
            positive_probs = probabilities[mask, 1]  # 选择正类概率
        else:
            positive_probs = probabilities[mask]  # 如果只有一个输出，假设已经是正类概率
        val_f1 = torch.mean(f1_score(torch.argmax(probabilities[mask],dim=1), labels[mask], num_classes=2)).cpu().numpy()
        auc_score = roc_auc_score(labels[mask].cpu().numpy(), positive_probs.cpu().numpy())

    return val_f1, auc_score

In [17]:
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import train_test_split
# 实例化模型
device = torch.device('cuda:1')
data = data.to(device)
model = GCN(num_features=data.x.shape[1], hidden_dim=64, num_classes=2, num_layers=2, activation=F.relu, dropout=0.5)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)


labeled_indices = np.where(node_labels != -1)[0]
print(labeled_indices)
num_labeled = len(labeled_indices)
num_train = int(num_labeled * 0.8)
num_test = num_labeled - num_train
print(num_test)

# 创建训练和测试掩码
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

train_mask[labeled_indices[:num_train]] = True
test_mask[labeled_indices[num_train:num_train+num_test]] = True
print(test_mask)

[47662 47676 47688 ... 61993 62002 62021]
276
tensor([False, False, False,  ..., False, False, False])


In [18]:
print(num_train)
print(train_mask)

1103
tensor([False, False, False,  ..., False, False, False])


In [19]:
print(data.x, data.y, data.edge_index)

tensor([[-1.1681, -0.3552,  0.2659,  ...,  0.2073,  0.0707,  0.9386],
        [-1.1859, -0.0988,  0.3882,  ..., -0.0512,  0.0488,  0.7805],
        [ 0.0633, -0.2000,  0.1515,  ..., -0.5243, -0.1393,  0.7616],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:1') tensor([-1, -1, -1,  ..., -1, -1, -1], device='cuda:1') tensor([[    0,     0,     1,  ..., 52521, 52521, 52521],
        [25549, 25552,  5429,  ..., 12575, 15622, 20221]], device='cuda:1')


In [20]:
num_epochs = 1000
for epoch in range(num_epochs):
    train_loss = train_model_scheduler(model, data.x, data.y, data.edge_index, optimizer, loss_fn, scheduler, train_mask)
    test_f1, test_auc = evaluate_model(model, data.x, data.y, data.edge_index, test_mask)
    
    if epoch % 10 == 0: 
        print(f'Epoch {epoch}: Train Loss:{train_loss:.4f}, Macro_F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')

Epoch 0: Train Loss:0.8268, Macro_F1: 0.3041, AUC_score: 0.4709
Epoch 10: Train Loss:0.6620, Macro_F1: 0.5822, AUC_score: 0.6620
Epoch 20: Train Loss:0.6329, Macro_F1: 0.6413, AUC_score: 0.6983
Epoch 30: Train Loss:0.5951, Macro_F1: 0.6367, AUC_score: 0.7200
Epoch 40: Train Loss:0.5702, Macro_F1: 0.6783, AUC_score: 0.7515
Epoch 50: Train Loss:0.5415, Macro_F1: 0.6756, AUC_score: 0.7742
Epoch 60: Train Loss:0.5161, Macro_F1: 0.6910, AUC_score: 0.7970
Epoch 70: Train Loss:0.4912, Macro_F1: 0.7110, AUC_score: 0.8079
Epoch 80: Train Loss:0.4765, Macro_F1: 0.7125, AUC_score: 0.8176
Epoch 90: Train Loss:0.4624, Macro_F1: 0.7079, AUC_score: 0.8211
Epoch 100: Train Loss:0.4566, Macro_F1: 0.7130, AUC_score: 0.8223
Epoch 110: Train Loss:0.4463, Macro_F1: 0.7295, AUC_score: 0.8236
Epoch 120: Train Loss:0.4371, Macro_F1: 0.7261, AUC_score: 0.8273
Epoch 130: Train Loss:0.4251, Macro_F1: 0.7341, AUC_score: 0.8362
Epoch 140: Train Loss:0.4178, Macro_F1: 0.7478, AUC_score: 0.8377
Epoch 150: Train Loss

In [21]:
num_epochs = 2000
for epoch in range(num_epochs):
    train_loss = train_model_scheduler(model, data.x, data.y, data.edge_index, optimizer, loss_fn, scheduler, train_mask)
    test_f1, test_auc = evaluate_model(model, data.x, data.y, data.edge_index, test_mask)
    
    if epoch % 10 == 0: 
        print(f'Epoch {epoch}: Train Loss:{train_loss:.4f}, Macro_F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')

Epoch 0: Train Loss:0.1302, Macro_F1: 0.7503, AUC_score: 0.8512
Epoch 10: Train Loss:0.1353, Macro_F1: 0.7636, AUC_score: 0.8520
Epoch 20: Train Loss:0.1359, Macro_F1: 0.7695, AUC_score: 0.8531
Epoch 30: Train Loss:0.1392, Macro_F1: 0.7597, AUC_score: 0.8537
Epoch 40: Train Loss:0.1441, Macro_F1: 0.7710, AUC_score: 0.8516
Epoch 50: Train Loss:0.1418, Macro_F1: 0.7576, AUC_score: 0.8532
Epoch 60: Train Loss:0.1454, Macro_F1: 0.7592, AUC_score: 0.8505
Epoch 70: Train Loss:0.1362, Macro_F1: 0.7576, AUC_score: 0.8535
Epoch 80: Train Loss:0.1312, Macro_F1: 0.7606, AUC_score: 0.8521
Epoch 90: Train Loss:0.1292, Macro_F1: 0.7619, AUC_score: 0.8527
Epoch 100: Train Loss:0.1332, Macro_F1: 0.7719, AUC_score: 0.8544
Epoch 110: Train Loss:0.1322, Macro_F1: 0.7580, AUC_score: 0.8524
Epoch 120: Train Loss:0.1326, Macro_F1: 0.7541, AUC_score: 0.8527
Epoch 130: Train Loss:0.1332, Macro_F1: 0.7572, AUC_score: 0.8509
Epoch 140: Train Loss:0.1281, Macro_F1: 0.7572, AUC_score: 0.8524
Epoch 150: Train Loss